In [3]:
import pandas as pd
import cryptpandas as crp

decrypted_df = crp.read_encrypted(path='encrypted_data/release_3547.crypt', password='oUFtGMsMEEyPCCP6')

decrypted_df.to_csv('decrypted_data/release_3547.csv')